In [50]:
from collections import namedtuple
from dataclasses import dataclass
from typing import List

In [22]:
def hex2bin(s):
    out = ""
    for c in s.lower():
        i = int(c, 16)
        #print(i)
        out += format(i, '0>4b')
    return out

hex2bin("D2FE28")

'110100101111111000101000'

In [86]:
@dataclass
class Packet:
    version: int
    typeid: int
    length: int
        
@dataclass
class Literal(Packet):
    number: int
        
@dataclass
class Operator(Packet):
    subpackets: List[Packet]
        
def take(s, n, i):
    return s[i:i+n], i+n

def takeint(s, n, i):
    t, i = take(s, n, i)
    return int(t, 2), i

class Stream:
    def __init__(self, hexstr):
        self.hexstr = hexstr
        self.binstr = hex2bin(hexstr)
        self.tree, self.extra = self.__parse_stream(0)
        
    def __parse_stream(self, i0=0):
        s = self.binstr
        i = i0
        v, i = takeint(s, 3, i)
        t, i = takeint(s, 3, i)
        if t == 4:  # Literal
            number = ""
            while True:
                group, i = take(s, 5, i)
                #print(group)
                number += group[1:]
                if group[0] == '0':
                    break
            return Literal(v, t, i-i0, int(number, 2)), i
        else: # Operator
            lengthid, i = takeint(s, 1, i)
            subpackets = []
            if lengthid == 0:
                l, i = takeint(s, 15, i)
                while l > 0:
                    sub, i = self.__parse_stream(i)
                    subpackets.append(sub)
                    l -= sub.length
            else:
                n, i = takeint(s, 11, i)
                subpackets = []
                for _ in range(n):
                    sub, i = self.__parse_stream(i)
                    subpackets.append(sub)
            return Operator(v, t, i-i0, subpackets), i              
    
    def show_tree(self):
        self.show_node(self.tree, "")
    
    def show_node(self, node, indent=""):
        if isinstance(node, Literal):
            print(indent+"Literal(version={0.version} typeid={0.typeid} number={0.number})".format(node))
        elif isinstance(node, Operator):
            print(indent+"Operator(version={0.version} typeid={0.typeid} npackets={1})".format(node, len(node.subpackets)))
            for s in node.subpackets:
                self.show_node(s, indent+"  ")
        else:
            pass
    
    def sum_version(self, node=None):
        if node is None:
            node = self.tree
        v = node.version
        if isinstance(node, Operator):
            for sub in node.subpackets:
                v += self.sum_version(sub)
        return v

def test(name, hexstr):
    print()
    print("-- {} --".format(name))
    s = Stream(hexstr)
    print(s.hexstr)
    print(s.binstr)
    s.show_tree()
    print(s.sum_version())

test("t1", "D2FE28")
test("t2", "38006F45291200")
test("t3", "EE00D40C823060")
test("t4", "8A004A801A8002F478")
test("t5", "620080001611562C8802118E34")
test("t6", "C0015000016115A2E0802F182340")
test("t7", "A0016C880162017C3686B18A3D4780")


-- t1 --
D2FE28
110100101111111000101000
Literal(version=6 typeid=4 number=2021)
6

-- t2 --
38006F45291200
00111000000000000110111101000101001010010001001000000000
Operator(version=1 typeid=6 npackets=2)
  Literal(version=6 typeid=4 number=10)
  Literal(version=2 typeid=4 number=20)
9

-- t3 --
EE00D40C823060
11101110000000001101010000001100100000100011000001100000
Operator(version=7 typeid=3 npackets=3)
  Literal(version=2 typeid=4 number=1)
  Literal(version=4 typeid=4 number=2)
  Literal(version=1 typeid=4 number=3)
14

-- t4 --
8A004A801A8002F478
100010100000000001001010100000000001101010000000000000101111010001111000
Operator(version=4 typeid=2 npackets=1)
  Operator(version=1 typeid=2 npackets=1)
    Operator(version=5 typeid=2 npackets=1)
      Literal(version=6 typeid=4 number=15)
16

-- t5 --
620080001611562C8802118E34
01100010000000001000000000000000000101100001000101010110001011001000100000000010000100011000111000110100
Operator(version=3 typeid=0 npackets=2)
  Operator(ve

In [88]:
inptext = ""
with open("input.txt", "r") as fh:
    for l in fh:
        inptext += l.strip()
inp = Stream(inptext)
inp.show_tree()
inp.sum_version()

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'